# 安裝必要套件

In [ ]:
!pip install -q torch torchvision matplotlib

import os
import zipfile
from google.colab import drive
from google.colab import files
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader

# 上傳並解壓縮資料集

In [ ]:
print("正在連接 Google 雲端硬碟...")
drive.mount('/content/drive')
print("Google 雲端硬碟連接完成。")
# 指定您的 .zip 檔案在雲端硬碟中的路徑
# 請將 'My Drive/your_dataset.zip' 替換為您實際的檔案路徑
zip_file_path_in_drive = '/content/drive/My Drive/dataset/topic4_release.zip'

# 指定解壓縮的目標資料夾名稱
extract_folder_name = "dataset"

# 檢查 zip 檔案是否存在
if os.path.exists(zip_file_path_in_drive):
    print(f"找到 zip 檔案：{zip_file_path_in_drive}")

    # 建立目標資料夾（如果不存在）
    if not os.path.exists(extract_folder_name):
        os.makedirs(extract_folder_name)

    # 解壓縮檔案
    print(f"正在解壓縮檔案到 {extract_folder_name}...")
    with zipfile.ZipFile(zip_file_path_in_drive, 'r') as zip_ref:
        zip_ref.extractall(extract_folder_name)
    print("解壓縮完成，開始讀取資料")
else:
    print(f"錯誤：找不到指定的 zip 檔案：{zip_file_path_in_drive}")
    print("請檢查檔案路徑是否正確，並確保檔案已上傳到 Google 雲端硬碟。")

print("解壓縮完成，開始讀取資料")

正在連接 Google 雲端硬碟...
Mounted at /content/drive
Google 雲端硬碟連接完成。
找到 zip 檔案：/content/drive/My Drive/dataset/topic4_release.zip
正在解壓縮檔案到 dataset...
解壓縮完成，開始讀取資料


NameError: name 'uploaded' is not defined

# 建立 Dataset

In [ ]:
class SRDataset(Dataset):
    def __init__(self, root_dir):
        self.hr_root = os.path.join(root_dir, "train", "High_Resolution")
        self.lr_root = os.path.join(root_dir, "train", "Low_Resolution")
        self.transform = T.ToTensor()

        # 遞迴找出所有 low-resolution 圖片
        self.lr_paths = []
        for root, _, files in os.walk(self.lr_root):
            for file in files:
                if file.endswith(".png"):
                    full_path = os.path.join(root, file)
                    self.lr_paths.append(full_path)

        # 依照路徑排序確保對齊（重要）
        self.lr_paths.sort()

    def __len__(self):
        return len(self.lr_paths)

    def __getitem__(self, idx):
        lr_path = self.lr_paths[idx]
        # 取得相對路徑以定位對應 HR 檔案
        rel_path = os.path.relpath(lr_path, self.lr_root)
        hr_path = os.path.join(self.hr_root, rel_path)

        # 載入圖片
        lr_img = Image.open(lr_path).convert("RGB")
        hr_img = Image.open(hr_path).convert("RGB")

        return self.transform(lr_img), self.transform(hr_img)

NameError: name 'Dataset' is not defined

# 定義 Upsampler CNN 模型

In [ ]:
class CNNUpsampler(nn.Module):
    def __init__(self, scale=2):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3 * scale * scale, 3, padding=1),
            nn.PixelShuffle(scale)
        )

    def forward(self, x):
        return self.model(x)

# 設定訓練參數

In [ ]:
epochs = 30
batch_size = 8
learning_rate = 1e-4
scale_factor = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = SRDataset("dataset")
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = CNNUpsampler(scale=scale_factor).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.L1Loss()

losses = []

# 開始訓練

In [ ]:
model.train()
for epoch in range(epochs):
    total_loss = 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    losses.append(avg_loss)
    print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")

    if(epoch % 5 == 0):
      torch.save(model.state_dict(), f"upsampler_{epoch}.pth")
      print(f"模型儲存為 upsampler_{epoch}.pth")

# 視覺化 Loss 曲線

In [ ]:
plt.plot(losses)
plt.xlabel("Epoch")
plt.ylabel("L1 Loss")
plt.title("Training Loss")
plt.grid(True)
plt.show()

# 儲存模型並下載

In [ ]:
torch.save(model.state_dict(), "upsampler.pth")
print("模型儲存為 upsampler.pth")

files.download("upsampler.pth")